In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, scale
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor ,GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn import decomposition, ensemble
import tensorflow 
import xgboost
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


Transforming Categorical Variables

In [4]:
encoder = LabelEncoder()
df["sex"] = encoder.fit_transform(df["sex"])#Famele--> 0   Male--> 1
df["smoker"] = encoder.fit_transform(df["smoker"])# Yes--> 1   No--> 0
df["region"] = encoder.fit_transform(df["region"])# Northeast--> 0   Nortwest--> 1  Southeast--> 2   Southwest--> 3


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
sex,1338.0,0.505232,0.500160,0.0000,0.00000,1.000,1.000000,1.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
smoker,1338.0,0.204783,0.403694,0.0000,0.00000,0.000,0.000000,1.00000
region,1338.0,1.515695,1.104885,0.0000,1.00000,2.000,2.000000,3.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [5]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df[["age","sex","bmi","children","smoker","region"]],
                                                                    df["charges"],test_size=0.2,random_state=42)

Min-Max Scaler

In [6]:
scaler = MinMaxScaler((-1,1))
x_minmax_train = scaler.fit_transform(x_train)
x_minmax_test = scaler.fit_transform(x_test)

Standard Scaler

In [7]:
s_scaler = StandardScaler()
x_stand_train = s_scaler.fit_transform(x_train)
x_stand_test = s_scaler.fit_transform(x_test)

PCA

In [8]:
pca = PCA()
x_pcamm_train = pca.fit_transform(scale(x_minmax_train))
x_pcamm_test = pca.fit_transform(scale(x_minmax_test))

x_pcasta_train = pca.fit_transform(scale(x_stand_train))
x_pcasta_test = pca.fit_transform(scale(x_stand_test))

In [10]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)# We see that all variable variances can be explained.

array([ 22.35,  41.21,  58.18,  73.72,  87.77, 100.01])

Linear Regression

In [37]:
lin = linear_model.LinearRegression()
lin_model_minmax = lin.fit(x_pcamm_train,y_train)
lin_model_stan = lin.fit(x_pcasta_train,y_train)

In [36]:
accuracy = model_selection.cross_val_score(lin_model_minmax, x_pcamm_test, y_test, cv = 10)
print("Linear Regression Min-Max Accuracy: %0.2f (+/- %0.2f)" % (accuracy.mean(), accuracy.std() * 2))

Linear Regression Min-Max Accuracy: 0.70 (+/- 0.36)


In [38]:
accuracy = model_selection.cross_val_score(lin_model_stan, x_pcasta_test, y_test, cv = 10)
print("Linear Regression Standard Accuracy: %0.2f (+/- %0.2f)" % (accuracy.mean(), accuracy.std() * 2))

Linear Regression Standard Accuracy: 0.70 (+/- 0.36)
